# Webscraping & Vectorization

In [157]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

## Webscraping AZLyrics for Frank Ocean Songs

In [75]:
''' 
Souping the discography page and storing track names, album and
    link to lyric page.
    
'''

# 3 ablums and their song count
albums_dct = [
    ('Nostalgia, Ultra', 11),
    ("The Lonny Breaux Collection", 62),
    ('Channel Orange', 17),
    ( 'Endless', 17),
    ('Blonde', 17),
    ('other', 99)
]

# dictionary will hold info about songs
songs_dct = {
    'track': [],
    'album': [],
    'lyrics': []
}

# Beautiful souping the discography page
discography_frank = 'https://www.azlyrics.com/f/frankocean.html'
soup = bs( requests.get(discography_frank).text, 'html5lib' )

all_divs = soup.find_all('div', attrs={'class':'listalbum-item'})
all_divs = [link.find('a') for link in all_divs]

In [79]:
def get_lyrics(track_link):
    soup = bs( requests.get(track_link).text, 'html5lib' )
    test = soup.find_all('div', attrs={'class': None, 'id': None})
    lyrics = ' '
    try:
        lyrics = test[0].text
    except IndexError:
        pass
    return lyrics

In [81]:
# extracting the information from each song

album_num = 0
track_num = 1
for div in all_divs:
    # track name
    songs_dct['track'].append(div.text)
    
    # album name
    if track_num > albums_dct[album_num][1]:
        track_num = 1
        
        print('Done with album: ', albums_dct[album_num][0])
        album_num += 1
      
    songs_dct['album'].append(albums_dct[album_num][0])
    track_num += 1
    
    #lyrics
    track_link = 'https://www.azlyrics.com' + div.get('href').replace('..', '')
    lyrics = get_lyrics(track_link)
    songs_dct['lyrics'].append(lyrics)
    
print('All done')    

Done with album:  Nostalgia, Ultra
Done with album:  The Lonny Breaux Collection
Done with album:  Channel Orange
Done with album:  Endless
Done with album:  Blonde


In [102]:
# Only keep first 5 albums
for category in songs_dct:
    songs_dct[category] = songs_dct[category][:124]
    
len(songs_dct['lyrics'])

## Cleaning the Text into a DF

In [153]:
import re
def preprocess(document):
    test = re.sub("\(.*\)", "", document)
    test = re.sub("\[.*\]", "", test)
    test = re.sub("[^\w\s]", "", test)
    test = test.replace('â', "").replace('\n', ' ').lower()
    return re.sub("[ ]+", " ", test)

In [161]:
updated_songs = songs_dct.copy()
updated_songs['lyrics'] = [preprocess(lyric) for lyric in updated_songs['lyrics']]
    
updated_songs['lyrics'][:3]

[' when we were kids we handpainted strawberries on a swing every moment was so precious then im still kicking it im daydreaming on a strawberry swing the entire earth is fighting all the world is at its end just in case an atom bomb comes falling on my lawn i should say and you should hear ive loved ive loved the good times here ive loved our good times here say hello then say farewell to the places you know we are all mortals arent we any moment this could go cry cry cry even though that wont change a thing but you should know you should hear that i have loved i have loved the good times here and i will miss our good times here spaceships are lifting off of a dying world and millions are left behind while the sky burns there wasnt room for you and i only you goodbye goodbye ',
 ' i think i started somethin i got what i wanted did didnt i cant feel nothin superhuman even when im fuckin viagra poppin every single record autotunin zero emotion muted emotion pitch corrected computed emot

In [159]:
df = pd.DataFrame(updated_songs)
df

,track,album,lyrics
0,Strawberry Swing,"Nostalgia, Ultra",when we were kids we handpainted strawberries...
1,Novacane,"Nostalgia, Ultra",i think i started somethin i got what i wante...
2,We All Try,"Nostalgia, Ultra",i believe jehovah jireh i believe theres heav...
3,Bitches Talkin' (Metal Gear Solid),"Nostalgia, Ultra",stop let me go you dont got no jodeci all you...
4,Songs For Women,"Nostalgia, Ultra",when i was younger i used to wonder like if i...
...,...,...,...
119,Close To You,Blonde,i will be honest i wasnt devastated but you c...
120,White Ferrari,Blonde,bad luck to talk on these rides mine on the r...
121,Seigfried,Blonde,the markings on your surface your speckled fa...
122,Godspeed,Blonde,i will always love you how i do let go of a p...


In [225]:
df.to_csv('Frank_Lyrics.csv', index=False)

!ls

Frank_Lyrics.csv               challenge_set_7_emmanuel.ipynb
Instagram_Scrape_Fail.ipynb    challenge_set_8_emmanuel.ipynb
Metis_2020_Submissions         decade_hits.csv
Pair_on_own.ipynb              exercise
Scraping_Azlyrics.ipynb        gitHub_blog
Stats_Prework                  model_selection.py
WebScrape                      top_100.pickle
__pycache__
